In [ ]:
!pip install -U adapter-transformers
!pip install optuna

In [ ]:
from datasets import load_dataset

scierc_name = 'nsusemiehl/SciERC'
scierc_dataset = load_dataset(scierc_name)
print(scierc_dataset.num_rows)

# acl_arc_name = 'zapsdcn/citation_intent'
# acl_arc_dataset = load_dataset(acl_arc_name)
# print(acl_arc_dataset.num_rows)



In [ ]:
scierc_dataset['train'][0]

In [ ]:
# Finding the number of labels
import numpy as np
labels = np.unique(np.array(scierc_dataset['train']['label']))
num_of_labels = labels.size

print(labels)
print(num_of_labels)

In [ ]:
# encoding the labels
def encode_labels(dataset):
    for i in range(num_of_labels):
        if dataset['label'] == labels[i]:
            dataset['label'] = i
    return dataset

scierc_dataset = scierc_dataset.map(encode_labels)
scierc_dataset['train'][0]

In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
scierc_dataset = scierc_dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
scierc_dataset = scierc_dataset.rename_column("label", 'labels')
# Transform to pytorch tensors and only output the required columns
scierc_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
from transformers import RobertaConfig
from transformers import RobertaAdapterModel

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=num_of_labels,
)
model = RobertaAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
# Add new adapter
model.add_adapter("sci_erc")
# Add a matching classification head
model.add_classification_head(
    "sci_erc",
    num_labels=num_of_labels,
    id2label={0:'COMPARE', 1:'CONJUNCTION', 2:'EVALUATE-FOR', 
            3:'FEATURE-OF', 4:'HYPONYM-OF', 5:'PART-OF', 6:'USED-FOR'}
)
# Activate the adapter
model.train_adapter("sci_erc")

In [ ]:
# We have to create a function to create a new model for hyperparameter tuning to work. This allows the tuner to create a new model to test on.
def model_init():
    config = RobertaConfig.from_pretrained(
        "roberta-base",
        num_labels=num_of_labels,
    )
    model = RobertaAdapterModel.from_pretrained(
        "roberta-base",
        config=config,
    )
    # Add new adapter
    model.add_adapter("sci_erc")
    # Add a matching classification head
    model.add_classification_head(
        "sci_erc",
        num_labels=num_of_labels,
        id2label={0:'COMPARE', 1:'CONJUNCTION', 2:'EVALUATE-FOR', 
                3:'FEATURE-OF', 4:'HYPONYM-OF', 5:'PART-OF', 6:'USED-FOR'}
    )
    # Activate the adapter
    model.train_adapter("sci_erc")
    return model

In [ ]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=50,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    logging_steps=10,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
)

# We want to minimize the loss function so we should not include f1.

# Adding Support for Tensorboard, supposedly you don't have to do this, but I find that it doesn't work
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback
writer = SummaryWriter()
writer = TensorBoardCallback(writer)

# This will throw a warning that we should not have both a model and a model_init, but adaptertrainer does not like it when we don't have a model.

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=scierc_dataset["train"],
    eval_dataset=scierc_dataset["validation"],
    model_init=model_init,
    callbacks=[writer]   
)

In [ ]:
# We are using optuna for tuning, by default it tries epochs, learning rate, and batch siz, we can try to give it more parameters to search but I feel like these might be the best for now.
# This will by default run a 100 trials.

trainer.hyperparameter_search(
    # hp_space = {Parameters to tune}
)

In [ ]:
trainer.evaluate(scierc_dataset['test'])